<a href="https://colab.research.google.com/github/nightmarewarrior/homework/blob/main/homework_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [26]:
!pip install tqdm
!pip install nltk
!pip install textdistance

In [27]:
import textdistance
import re
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

from tqdm import tqdm
from string import punctuation

In [28]:
import re
import textdistance
from tqdm.notebook import tqdm
from string import punctuation
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [29]:
!unzip '/content/data.zip'

Archive:  /content/data.zip
replace correct_sents.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sents_with_mistakes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wiki_data.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/._wiki_data.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [30]:
corpus = open('wiki_data.txt', encoding='utf8').read()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()

In [31]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [32]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

In [33]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

In [40]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()

    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)

    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn * 4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    sort_metric = max(word[1] for word in closest)
    closest_dict = {word: P(word) for word, metr in closest if metr == sort_metric}

    max_word = max(closest_dict, key=closest_dict.get, default=None)
    return max_word

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [41]:
print (get_closest_hybrid_match('сонце', X, vec))

солнце


In [42]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]

        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

Оценка

In [43]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7928964482241121
0.0015527950310559005
0.09004249454461927


## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!)

3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [44]:
corpus = open('wiki_data.txt', encoding='utf8').read()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()

In [45]:
vocab = Counter(re.findall('\w+', corpus.lower()))

In [53]:
def generate_deletes(word):

    return list(set(word[:i] + word[i+1:] for i in range(len(word))))


def build_deletion_dictionary(vocab, max_distance=1):

    deletion_dict = {}
    for word in vocab:
        for deleted_word in generate_deletes(word):
            if deleted_word in deletion_dict:
                deletion_dict[deleted_word].append(word)
            else:
                deletion_dict[deleted_word] = [word]
    return deletion_dict


def symspell_correct(word, deletion_dict, vocab):

    if word in vocab:
        return word

    candidates = []
    for deleted_word in generate_deletes(word):
        if deleted_word in deletion_dict:
            candidates.extend(deletion_dict[deleted_word])

    if not candidates:
        return word

    candidate_scores = {candidate: (vocab[candidate], len(candidate)) for candidate in set(candidates)}
    sorted_candidates = sorted(candidate_scores.items(), key=lambda x: (-x[1][0], x[1][1]))

    return sorted_candidates[0][0]

def correct_sentences(sentences, vocab, deletion_dict):
    corrected_sentences = []
    for sentence in sentences:
        corrected_words = []
        for word in re.findall(r'\w+', sentence):
            corrected_word = symspell_correct(word, deletion_dict, vocab)
            corrected_words.append(corrected_word)
        corrected_sentences.append(' '.join(corrected_words))
    return corrected_sentences


In [54]:
print (get_closest_hybrid_match('сонце', X, vec))

солнце


In [55]:
%time
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]

        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.72 µs


  0%|          | 0/915 [00:00<?, ?it/s]

In [56]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7928964482241121
0.0015527950310559005
0.09004249454461927


Особо лучше почему-то не стало... Возможно, я делаю что-то не так, но задерживая это задание ещё дольше как будто бы ни к чему новому не прихожу...